# ELE2769 -  Remote Sensing Image Analysis - Assignment 1

## 1st Problem - Image Pan-Sharpening Correction

### Introduction

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

Fusce tempor ipsum quis rhoncus volutpat. In molestie tortor id tristique condimentum. Nulla id erat vitae velit efficitur bibendum. Duis blandit at erat ac vehicula. Quisque eu purus in orci egestas commodo in nec metus. Donec pulvinar fermentum enim feugiat ornare. Pellentesque cursus imperdiet porttitor. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed semper, velit at dignissim dapibus, nisl turpis rhoncus massa, in consequat magna magna ac dolor. Aliquam vitae mattis leo, eu egestas nulla. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Aliquam mauris nibh, imperdiet id malesuada a, consectetur non sapien. Ut ultricies eros augue, ac pellentesque quam luctus mollis.

Nullam ultricies, sapien sed tempus eleifend, dui lorem pellentesque orci, sit amet ultricies odio quam ut nisi. Vestibulum cursus, turpis id euismod porta, elit metus posuere mauris, eu bibendum tortor eros sed augue. Nam eu tortor diam. Nullam et arcu vitae ligula dapibus ultrices. Quisque nisi tellus, fringilla ac auctor nec, ullamcorper nec enim. Ut porttitor commodo gravida. Donec facilisis est in justo vulputate, at facilisis odio maximus. Praesent auctor auctor odio eget efficitur. Morbi molestie sem vel feugiat posuere. Fusce ultrices iaculis lorem, sed mollis nibh egestas sed. Aliquam volutpat a quam nec imperdiet. 

### Notebook Setup:

In [ ]:
%matplotlib notebook
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA

In [ ]:
save_figs = False
dpi = 200

### Defining the functions we'll need

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

In [ ]:
def multichannel(chn1, chn2, chn3):
    '''This function takes in 3 same sized channels and puts them together in a single image'''
    
    if chn1.shape == chn2.shape == chn3.shape:
        im_out = np.zeros(chn1.shape + (3,))
        im_out[:,:,0] = chn1
        im_out[:,:,1] = chn2
        im_out[:,:,2] = chn3
        
        return im_out
    
    else:
        print('The Channels Must be The Same Size')

In [ ]:
def pansharpening_HSV(im_in, im_pan):
    '''This function takes the RGB image, converts it to HSV and substitutes the V by the Pan band value of a same sized image'''
    
    if im_in.shape[:2] == im_pan.shape:
        im_in = im_in
        im_pan = im_pan
        img_HSV_in = cv2.cvtColor(im_in.astype('uint8'), cv2.COLOR_RGB2HSV)
        img_HSV_in[:,:,2] = im_pan
        img_HSV_out = cv2.cvtColor(img_HSV_in.astype('uint8'), cv2.COLOR_HSV2RGB)

        return img_HSV_out
    
    else:
        print('The X Y Sizes Should be the Same')

In [ ]:
def pansharpening_PCA(im_in, im_pan, ms=True, report=False):
    '''This function takes the input image (RGB), executes a PCA analysis and substitutes the PC1 by the Pan band value of a same sized image. 
    The ms input=True will adjust the mean and standard deviation of the Pan data to that of the PC1.
    The report mode returns the PC1 and pan linear arrays'''

    
    
    if im_in.shape[:2] == im_pan.shape:
        new_shape = (im_in.shape[0]*im_in.shape[1],3)
        pan_newshape = im_pan.reshape(new_shape[0])
        
        
        pca = PCA(n_components=2)
        pcomponents = pca.fit_transform(im_in.reshape(new_shape))
        
        if ms == True:
            adjusted_pan = adjust_dist(pan_newshape, pcomponents[:,0])
        else:
            adjusted_pan = pan_newshape
        
        pcomponents[:,0] = adjusted_pan
        
        
        im_out = pca.inverse_transform(pcomponents)
        im_out = im_out.reshape(im_in.shape)
        
        if report == False:
            return im_out
        else:
            return pcomponents[:,0], pan_newshape
    
    else:
        print('The X Y Sizes Should be the Same')

In [ ]:
def adjust_dist(in_dist, out_dist, report=False):
    '''This function takes in a distribution and adjusts its std and mean to that of the out distribution
    The report mode outputs the std and mean of the adjusted distribution'''
    adjusted_dist = np.float64(np.mean(out_dist) + (in_dist - np.mean(in_dist))*(np.std(out_dist)/np.std(in_dist)))
    
    if report==True:
        mean = np.mean(adjusted_dist)
        std = np.std(adjusted_dist)
        print("The mean is {}, the std is {}".format(mean, std))
        return adjusted_dist
    else:
        return adjusted_dist

### Loading the raw images

**Loading the data. As OpenCV and matplotlib deals with greyscale images differently (0-255 for openCV and 0-1 for mplt), all images read with mpl are multiplied by 255 to account for this difference and prevent any problems**

In [ ]:
img_preffix = 'img_raw_'
img_raw = ['red', 'grn', 'blu', 'nir', 'pan']
for i in range(len(img_raw)):
    var = img_preffix + img_raw[i]
    vars()[var] = plt.imread('sjc_'+img_raw[i]+'.png')*255

### PAN vs Single bands

In [ ]:
fig,((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (8,8))

axes = [ax1, ax2, ax3, ax4, ax5, ax6]
data = [img_raw_red, img_raw_grn, img_raw_blu, img_raw_nir, img_raw_pan]
titles = ['Red', 'Green', 'Blue', 'NIR', 'Pan']

for i in range(5):

    axes[i].imshow(data[i])
    axes[i].set_axis_off()
    axes[i].set_title(titles[i]+' Channel')

ax6.axis('off')
fig.tight_layout()
fig.suptitle('Comparing R,G,B and Pan band resolution ')


if save_figs == True:
    plt.savefig('raw_channels.tiff', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

Putting the Channels together to into 2 pictures (R,G,B) and (NIR,R,G)

### Comparing RGB with NIR RG

In [ ]:
img_rgb = multichannel(img_raw_red,img_raw_grn,img_raw_blu)
img_nrg = multichannel(img_raw_nir,img_raw_red,img_raw_blu)

In [ ]:
fig,(ax1, ax2) = plt.subplots(1,2, figsize = (8,8))

axes = [ax1, ax2]
data = [img_rgb, img_nrg]
titles = ['RGB', 'NIR RG']

for i in range(2):

    axes[i].imshow(data[i].astype('uint8'))
    axes[i].set_axis_off()
    axes[i].set_title('Unscaled '+titles[i]+' Channel')

fig.tight_layout()
fig.suptitle('Comparing RGB vs NIR RG band composition')
if save_figs == True:
    plt.savefig('compose_channels.tiff', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

Resizing the images to the same size as PAN using 3 different interpolation methods: nearest neighborhood, bilinear, bicubic

### Comparing different interpolation methods

In [ ]:
img_rgb_resize_nn = cv2.resize((img_rgb),img_raw_pan.shape,interpolation = cv2.INTER_NEAREST)
img_rgb_resize_bl = cv2.resize((img_rgb),img_raw_pan.shape,interpolation = cv2.INTER_LINEAR)
img_rgb_resize_bc = cv2.resize((img_rgb),img_raw_pan.shape,interpolation = cv2.INTER_CUBIC)

In [ ]:
img_nrg_resize_nn = cv2.resize((img_nrg),img_raw_pan.shape,interpolation = cv2.INTER_NEAREST)
img_nrg_resize_bl = cv2.resize((img_nrg),img_raw_pan.shape,interpolation = cv2.INTER_LINEAR)
img_nrg_resize_bc = cv2.resize((img_nrg),img_raw_pan.shape,interpolation = cv2.INTER_CUBIC)

In [ ]:
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (8,8))

ax1.imshow(img_rgb_resize_nn.astype('uint8'))
ax1.set_axis_off()
ax1.set_title('Rescaled RGB Image \n Nearest Neighborhood Method', fontsize = 8)

ax3.imshow(img_rgb_resize_bl.astype('uint8'))
ax3.set_axis_off()
ax3.set_title('Rescaled RGB Image \n Bilinear Method', fontsize = 8)

ax5.imshow(img_rgb_resize_bc.astype('uint8'))
ax5.set_axis_off()
ax5.set_title('Rescaled RGB Image \n Bicubic Method', fontsize = 8)

ax2.imshow(img_nrg_resize_nn.astype('uint8'))
ax2.set_axis_off()
ax2.set_title('Rescaled NIR RG Image \n Nearest Neighborhood Method', fontsize = 8)

ax4.imshow(img_nrg_resize_bl.astype('uint8'))
ax4.set_axis_off()
ax4.set_title('Rescaled NIR RG Image \n Bilinear Method', fontsize = 8)


ax6.imshow(img_nrg_resize_bc.astype('uint8'))
ax6.set_axis_off()
ax6.set_title('Rescaled NIR RG Image \n Bicubic Method', fontsize = 8)

fig.suptitle('The effect of interpolation on image rescaling')
fig.tight_layout()


if save_figs == True:
    plt.savefig('rescale.tiff', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

### Pan-Sharpening HSV in all interpolation methods

In [ ]:
img_Pan_rgb_nn = pansharpening_HSV(img_rgb_resize_nn,img_raw_pan)
img_Pan_rgb_bl = pansharpening_HSV(img_rgb_resize_bl,img_raw_pan)
img_Pan_rgb_bc = pansharpening_HSV(img_rgb_resize_bc,img_raw_pan)

In [ ]:
img_Pan_nrg_nn = pansharpening_HSV(img_nrg_resize_nn,img_raw_pan)
img_Pan_nrg_bl = pansharpening_HSV(img_nrg_resize_bl,img_raw_pan)
img_Pan_nrg_bc = pansharpening_HSV(img_nrg_resize_bc,img_raw_pan)

In [ ]:
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (8,8))

ax1.imshow(img_Pan_rgb_nn)
ax1.set_axis_off()
ax1.set_title('HSV Pan-Sharpened RGB Image \n Nearest Neighborhood Method', fontsize = 8)

ax3.imshow(img_Pan_rgb_bl)
ax3.set_axis_off()
ax3.set_title('HSV Pan-Sharpened RGB Image \n Bilinear Method', fontsize = 8)

ax5.imshow(img_Pan_rgb_bc)
ax5.set_axis_off()
ax5.set_title('HSV Pan-Sharpened RGB Image \n Bicubic Method', fontsize = 8)

ax2.imshow(img_Pan_nrg_nn)
ax2.set_axis_off()
ax2.set_title('HSV Pan-Sharpened NIR RG Image \n Nearest Neighborhood Method', fontsize = 8)

ax4.imshow(img_Pan_nrg_bl)
ax4.set_axis_off()
ax4.set_title('HSV Pan-Sharpened NIR RG Image \n Bilinear Method', fontsize = 8)


ax6.imshow(img_Pan_nrg_bc)
ax6.set_axis_off()
ax6.set_title('HSV Pan-Sharpened NIR RG Image \n Bicubic Method', fontsize = 8)

fig.suptitle('The effect of interpolation on image Pan_Sharpening')
fig.tight_layout()


if save_figs == True:
    plt.savefig('hsi_pansharp.tiff', dpi=dpi)

### Adjusting the mean and std

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

Consider a distribution $(x_i)$ with mean $m_1$ and standard deviation $s_1$ that we want to approximate to the mean and standard deviation of a second distribution, $m_2$ and $s_2$ respectively. The adjusted distribution can be calculated as:

$$y_i = m_2 + (x_i - m_i)*\frac{s_2}{s_1}$$

In [ ]:
pc1, pan = pansharpening_PCA(img_rgb_resize_nn, img_raw_pan, report=True)

In [ ]:
adjusted = adjust_dist(pan, pc1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1)
ax1.hist(pc1)
ax1.set_xlim(-200,200)
ax1.set_title('PC1 distribution')
ax2.hist(pan, color='r')
ax2.set_xlim(-200,200)
ax2.set_title('Pan distribution')
ax3.hist(adjusted, color='g')
ax3.set_xlim(-200,200)
ax3.set_title('Adjusted Pan distribution')

fig.suptitle('The effect of mean and STD adjustment')
fig.tight_layout()


if save_figs == True:
    plt.savefig('hsi_pansharp.tiff', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

### The effect of mean and std adjustment in the final PCA Pan-Sharped image

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

In [ ]:
pan_noadjust = pansharpening_PCA(img_rgb_resize_nn,img_raw_pan, ms=False)
pan_adjust = pansharpening_PCA(img_rgb_resize_nn,img_raw_pan)

fig, axis = plt.subplots(1,2)
axis[0].imshow(pan_noadjust.astype('uint8'))
axis[0].set_title('No Adjustment', fontsize=10)
axis[1].imshow(pan_adjust.astype('uint8'))
axis[1].set_title('Adjusted distribution')
fig.suptitle('The effect of distribution adjustment in PCA Pan Sharpening', fontsize=10)
fig.tight_layout()

if save_figs == False:
    plt.savefig('magicspell.png', dpi=dpi)

### PCA pan sharpening - all interpolationmethods

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

In [ ]:
img_PanPCA_rgb_nn = pansharpening_PCA(img_rgb_resize_nn,img_raw_pan)
img_PanPCA_rgb_bl = pansharpening_PCA(img_rgb_resize_bl,img_raw_pan)
img_PanPCA_rgb_bc = pansharpening_PCA(img_rgb_resize_bc,img_raw_pan)

In [ ]:
img_PanPCA_nrg_nn = pansharpening_PCA(img_nrg_resize_nn,img_raw_pan)
img_PanPCA_nrg_bl = pansharpening_PCA(img_nrg_resize_bl,img_raw_pan)
img_PanPCA_nrg_bc = pansharpening_PCA(img_nrg_resize_bc,img_raw_pan)

In [ ]:
fig, ((ax1, ax2),(ax3, ax4),(ax5, ax6)) = plt.subplots(3,2, figsize = (8,8))

ax1.imshow(img_PanPCA_rgb_nn.astype('uint8'))
ax1.set_axis_off()
ax1.set_title('PCA Pan-Sharpened RGB Image \n Nearest Neighborhood Method', fontsize = 8)

ax3.imshow(img_PanPCA_rgb_bl.astype('uint8'))
ax3.set_axis_off()
ax3.set_title('PCA Pan-Sharpened RGB Image \n Bilinear Method', fontsize = 8)

ax5.imshow(img_PanPCA_rgb_bc.astype('uint8'))
ax5.set_axis_off()
ax5.set_title('PCA Pan-Sharpened RGB Image \n Bicubic Method', fontsize = 8)

ax2.imshow(img_PanPCA_nrg_nn.astype('uint8'))
ax2.set_axis_off()
ax2.set_title('PCA Pan-Sharpened NIR RG Image \n Nearest Neighborhood Method', fontsize = 8)

ax4.imshow(img_PanPCA_nrg_bl.astype('uint8'))
ax4.set_axis_off()
ax4.set_title('PCA Pan-Sharpened NIR RG Image \n Bilinear Method', fontsize = 8)


ax6.imshow(img_PanPCA_nrg_bc.astype('uint8'))
ax6.set_axis_off()
ax6.set_title('PCA Pan-Sharpened NIR RG Image \n Bicubic Method', fontsize = 8)

fig.tight_layout()

if save_figs == True:
    plt.savefig('pca_pansharp.tiff', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

### Comparing PCA to HSV pan sharpening

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

In [ ]:
HSV_pan = pansharpening_HSV(img_rgb_resize_nn,img_raw_pan)
PCA_pan = pansharpening_PCA(img_rgb_resize_nn,img_raw_pan)

fig, axis = plt.subplots(1,2)
axis[0].imshow(HSV_pan)
axis[0].set_title('HSV')
axis[1].imshow(PCA_pan.astype('uint8'))
axis[1].set_title('PCA')
fig.suptitle('HSV vs PCA Pan Sharpening \n Nearest Neibourhood Interpolation')

if save_figs == True:
    plt.savefig('hsivsmagicspall', dpi=dpi)

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

## Conclusion

## 2nd Problem - Georeferencing for area estimation over time

### Introduction

 Lorem ipsum dolor sit amet, consectetur adipiscing elit. Ut laoreet ex non quam cursus mollis. Pellentesque tempor vestibulum massa vitae condimentum. Ut luctus magna quis nisl gravida, at tincidunt odio mollis. Donec scelerisque posuere orci at rutrum. Nulla auctor massa ac nulla condimentum, at faucibus quam mattis. Duis pellentesque magna sit amet lorem congue volutpat. Cras sed tortor quis lorem euismod finibus maximus vel urna. Duis euismod enim faucibus mauris pretium vestibulum. Maecenas ut arcu ex.

Fusce tempor ipsum quis rhoncus volutpat. In molestie tortor id tristique condimentum. Nulla id erat vitae velit efficitur bibendum. Duis blandit at erat ac vehicula. Quisque eu purus in orci egestas commodo in nec metus. Donec pulvinar fermentum enim feugiat ornare. Pellentesque cursus imperdiet porttitor. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed semper, velit at dignissim dapibus, nisl turpis rhoncus massa, in consequat magna magna ac dolor. Aliquam vitae mattis leo, eu egestas nulla. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Aliquam mauris nibh, imperdiet id malesuada a, consectetur non sapien. Ut ultricies eros augue, ac pellentesque quam luctus mollis.

Nullam ultricies, sapien sed tempus eleifend, dui lorem pellentesque orci, sit amet ultricies odio quam ut nisi. Vestibulum cursus, turpis id euismod porta, elit metus posuere mauris, eu bibendum tortor eros sed augue. Nam eu tortor diam. Nullam et arcu vitae ligula dapibus ultrices. Quisque nisi tellus, fringilla ac auctor nec, ullamcorper nec enim. Ut porttitor commodo gravida. Donec facilisis est in justo vulputate, at facilisis odio maximus. Praesent auctor auctor odio eget efficitur. Morbi molestie sem vel feugiat posuere. Fusce ultrices iaculis lorem, sed mollis nibh egestas sed. Aliquam volutpat a quam nec imperdiet. 

### Methodology

Fusce tempor ipsum quis rhoncus volutpat. In molestie tortor id tristique condimentum. Nulla id erat vitae velit efficitur bibendum. Duis blandit at erat ac vehicula. Quisque eu purus in orci egestas commodo in nec metus. Donec pulvinar fermentum enim feugiat ornare. Pellentesque cursus imperdiet porttitor. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed semper, velit at dignissim dapibus, nisl turpis rhoncus massa, in consequat magna magna ac dolor. Aliquam vitae mattis leo, eu egestas nulla. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Aliquam mauris nibh, imperdiet id malesuada a, consectetur non sapien. Ut ultricies eros augue, ac pellentesque quam luctus mollis.

Nullam ultricies, sapien sed tempus eleifend, dui lorem pellentesque orci, sit amet ultricies odio quam ut nisi. Vestibulum cursus, turpis id euismod porta, elit metus posuere mauris, eu bibendum tortor eros sed augue. Nam eu tortor diam. Nullam et arcu vitae ligula dapibus ultrices. Quisque nisi tellus, fringilla ac auctor nec, ullamcorper nec enim. Ut porttitor commodo gravida. Donec facilisis est in justo vulputate, at facilisis odio maximus. Praesent auctor auctor odio eget efficitur. Morbi molestie sem vel feugiat posuere. Fusce ultrices iaculis lorem, sed mollis nibh egestas sed. Aliquam volutpat a quam nec imperdiet. 

### Results

Fusce tempor ipsum quis rhoncus volutpat. In molestie tortor id tristique condimentum. Nulla id erat vitae velit efficitur bibendum. Duis blandit at erat ac vehicula. Quisque eu purus in orci egestas commodo in nec metus. Donec pulvinar fermentum enim feugiat ornare. Pellentesque cursus imperdiet porttitor. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed semper, velit at dignissim dapibus, nisl turpis rhoncus massa, in consequat magna magna ac dolor. Aliquam vitae mattis leo, eu egestas nulla. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Aliquam mauris nibh, imperdiet id malesuada a, consectetur non sapien. Ut ultricies eros augue, ac pellentesque quam luctus mollis.

Nullam ultricies, sapien sed tempus eleifend, dui lorem pellentesque orci, sit amet ultricies odio quam ut nisi. Vestibulum cursus, turpis id euismod porta, elit metus posuere mauris, eu bibendum tortor eros sed augue. Nam eu tortor diam. Nullam et arcu vitae ligula dapibus ultrices. Quisque nisi tellus, fringilla ac auctor nec, ullamcorper nec enim. Ut porttitor commodo gravida. Donec facilisis est in justo vulputate, at facilisis odio maximus. Praesent auctor auctor odio eget efficitur. Morbi molestie sem vel feugiat posuere. Fusce ultrices iaculis lorem, sed mollis nibh egestas sed. Aliquam volutpat a quam nec imperdiet. 

### Conclusion

Fusce tempor ipsum quis rhoncus volutpat. In molestie tortor id tristique condimentum. Nulla id erat vitae velit efficitur bibendum. Duis blandit at erat ac vehicula. Quisque eu purus in orci egestas commodo in nec metus. Donec pulvinar fermentum enim feugiat ornare. Pellentesque cursus imperdiet porttitor. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed semper, velit at dignissim dapibus, nisl turpis rhoncus massa, in consequat magna magna ac dolor. Aliquam vitae mattis leo, eu egestas nulla. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Aliquam mauris nibh, imperdiet id malesuada a, consectetur non sapien. Ut ultricies eros augue, ac pellentesque quam luctus mollis.

Nullam ultricies, sapien sed tempus eleifend, dui lorem pellentesque orci, sit amet ultricies odio quam ut nisi. Vestibulum cursus, turpis id euismod porta, elit metus posuere mauris, eu bibendum tortor eros sed augue. Nam eu tortor diam. Nullam et arcu vitae ligula dapibus ultrices. Quisque nisi tellus, fringilla ac auctor nec, ullamcorper nec enim. Ut porttitor commodo gravida. Donec facilisis est in justo vulputate, at facilisis odio maximus. Praesent auctor auctor odio eget efficitur. Morbi molestie sem vel feugiat posuere. Fusce ultrices iaculis lorem, sed mollis nibh egestas sed. Aliquam volutpat a quam nec imperdiet. 